# Task B: Meta-Learning Perfomance Prediction

In this task, you will use information on training parameters and metadata on multiple OpenML dataset to train a performance predictor that performs well even for unseen datasets. You are provided with config parameters and metafeatures for six datasets. The datasets are split into training datasets and test datasets and you should only train on the training datasets.

For questions, you can contact zimmerl@informatik.uni-freiburg.

__Note: Please use the dataloading and splits you are provided with in this notebook.__

## Specifications:

* Data: six_datasets_lw.json
* Number of datasets: 6
* Training datasets: higgs, vehicle, adult, volkert
* Test datasets: Fashion-MNIST, jasmine
* Number of configurations: 2000
* Available data: architecture parameters and hyperparameters, metafeatures 
* Target: final validation accuracy
* Evaluation metric: MSE

## Importing and splitting data

Note: There are 51 steps logged, 50 epochs plus the 0th epoch, prior to any weight updates.

In [1]:
%%capture
%cd ..
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from api import Benchmark

In [2]:
bench_dir = "data/six_datasets_lw.json"
bench = Benchmark(bench_dir, cache=False)

==> Loading data...
==> No cached data found or cache set to False.
==> Reading json data...
==> Done.


In [4]:
with open("data/metafeatures.json", "r") as f:
    metafeatures = json.load(f)

In [57]:
# Dataset split
dataset_names = bench.get_dataset_names()
print(dataset_names)

train_datasets = ['adult', 'higgs', 'vehicle', 'volkert']
test_datasets = ['Fashion-MNIST', 'jasmine']

['Fashion-MNIST', 'adult', 'higgs', 'jasmine', 'vehicle', 'volkert']


In [58]:
# Prepare data
def read_data(datasets):
    n_configs = bench.get_number_of_configs(datasets[0])
    data = [bench.query(dataset_name=d, tag="Train/val_accuracy", config_id=ind) for d in datasets for ind in range(n_configs)]
    configs = [bench.query(dataset_name=d, tag="config", config_id=ind) for d in datasets for ind in range(n_configs)]
    dataset_names = [d for d in datasets for ind in range(n_configs)]
    
    y = np.array([curve[-1] for curve in data])
    return np.array(configs), y, np.array(dataset_names)

class TrainValSplitter():
    """Splits 25 % data as a validation split."""
    
    def __init__(self, dataset_names):
        self.ind_train, self.ind_val = train_test_split(np.arange(len(X)), test_size=0.25, stratify=dataset_names)
        
    def split(self, a):
        return a[self.ind_train], a[self.ind_val]

X, y, dataset_names = read_data(train_datasets)
X_test, y_test, dataset_names_test = read_data(test_datasets)

tv_splitter = TrainValSplitter(dataset_names=dataset_names)

X_train, X_val = tv_splitter.split(X)
y_train, y_val = tv_splitter.split(y)
dataset_names_train, dataset_names_val = tv_splitter.split(dataset_names)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("X_val:", X_val.shape)

X_train: (6000,)
X_test: (4000,)
X_val: (2000,)


The data contains the configuration.

__Note__: Not all parameters vary across different configurations. The varying parameters are batch_size, max_dropout, max_units, num_layers, learning_rate, momentum, weight_decay

In [59]:
# Take a look at one datapoint
datapoint_id = 1
config = X_train[datapoint_id]
dataset_name = dataset_names_train[datapoint_id]
example_metafeature = metafeatures[dataset_name]["NumberOfClasses"]

print("Config example:", X_train[0], sep="\n")
print("\nMeta-feature 'Number of classes':", example_metafeature)

Config example:
{'batch_size': 18, 'imputation_strategy': 'mean', 'learning_rate_scheduler': 'cosine_annealing', 'loss': 'cross_entropy_weighted', 'network': 'shapedmlpnet', 'max_dropout': 0.49544085228434, 'normalization_strategy': 'standardize', 'optimizer': 'sgd', 'cosine_annealing_T_max': 50, 'cosine_annealing_eta_min': 1e-08, 'activation': 'relu', 'max_units': 194, 'mlp_shape': 'funnel', 'num_layers': 3, 'learning_rate': 0.00022790650718781426, 'momentum': 0.10405150388775633, 'weight_decay': 0.07421575634467908}

Meta-feature 'Number of classes': 2.0


In [60]:
# Look at some metafeatures
iterator = iter(metafeatures[dataset_name].items())
for ind in range(5):
    feature, value = iterator.__next__()
    print(feature, value)

AutoCorrelation 0.634425994553756
ClassEntropy 0.7938438393644257
Dimensionality 0.00030711273084640267
EquivalentNumberOfAtts 11.068507517484338
MajorityClassPercentage 76.07182343065395


### Data Analysis


In [67]:
# Take a look at one datapoint
datapoint_id = 23
config = X_train[datapoint_id]
dataset_name = dataset_names_train[datapoint_id]
example_metafeature = metafeatures[dataset_name]["NumberOfClasses"]

print("Config example:", X_train[0], sep="\n")
#print("\nMeta-feature 'Number of classes':", example_metafeature)
print()
print(len(metafeatures[dataset_name]))

Config example:
{'batch_size': 18, 'imputation_strategy': 'mean', 'learning_rate_scheduler': 'cosine_annealing', 'loss': 'cross_entropy_weighted', 'network': 'shapedmlpnet', 'max_dropout': 0.49544085228434, 'normalization_strategy': 'standardize', 'optimizer': 'sgd', 'cosine_annealing_T_max': 50, 'cosine_annealing_eta_min': 1e-08, 'activation': 'relu', 'max_units': 194, 'mlp_shape': 'funnel', 'num_layers': 3, 'learning_rate': 0.00022790650718781426, 'momentum': 0.10405150388775633, 'weight_decay': 0.07421575634467908}

62


In [53]:
for datapoint_id in range(len(X_train)):
    config = X_train[datapoint_id]
    dataset_name = dataset_names_train[datapoint_id].item()
    if dataset_name == 'vehicle':
        for _ in range(len(metafeatures['higgs'])):
            print(type(metafeatures[dataset_name]))
            metafeatures[dataset_name].item() = metafeatures[dataset_name].item().pop()

SyntaxError: can't assign to function call (<ipython-input-53-e167ed45bbf9>, line 7)

In [90]:
metadata_new = {}
train_datasets = ['adult', 'higgs', 'volkert']
for i in enumerate(train_datasets):
    data = []
    print(type(metafeatures[i[1]]))
    for k, v in metafeatures[i[1]].items():
        data.append(v)
    metadata_new[i[1]] = data
print(metadata_new)
np.concatenate((data))

<class 'dict'>
<class 'dict'>
<class 'dict'>
{'adult': [0.634425994553756, 0.7938438393644257, 0.00030711273084640267, 11.068507517484338, 76.07182343065395, 37155.0, 3.44192266924963, 152.69309629815925, 189664.13459727284, 0.16542318099233, 41.0, 11.89465899659272, 105604.02542315713, 1.7809891200338273, 30.359637681213712, 31819.974765570616, 0.0717209468494675, 23.832203118732952, 11.222222222222221, 3.063860808334838, 18914.620326608216, 0.795215031650176, -0.18426874062378573, 10.078088530363212, 0.00818704228545, 2.0, -0.31652485666094055, 2.5709727555918307, 23.928176569346054, 11687.0, 2.0, 2.0, 15.0, 48842.0, 3620.0, 6465.0, 6.0, 9.0, 13.333333333333334, 7.411653904426519, 0.8824372466319971, 40.0, 60.0, 0.8343198263526672, 0.42324176943424885, 31.502211211662093, 0.01068033823523, 0.09993102873665205, 9.936326207089905, 1.6008102873649352, 4.504453651154136, 63.96234797919819, 0.0623734340938, 0.9982360975676194, 208.3575310294492, 2.736773252802121, 53.18403354052852, 48225

ValueError: zero-dimensional arrays cannot be concatenated

## Training and scoring

In [8]:
class ConstantPerformancePredictor():
    """A predictor that predicts the mean of the performances seen on the training data."""
    
    def __init__(self):
        self.constant_prediction = 0
        
    def fit(self, X, y, dataset_names, metafeatures):
        self.constant_prediction = np.mean(y)
    
    def predict(self, X):
        predictions = [self.constant_prediction] * len(X)
        return predictions
    
def score(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

In [9]:
# Train and validate
predictor = ConstantPerformancePredictor()
predictor.fit(X_train, y_train, dataset_names_train, metafeatures)
preds = predictor.predict(X_val)
mse = score(y_val, preds)
print(mse)

283.5325483378039


In [10]:
# Final evaluation
final_preds = predictor.predict(X_test)
final_score = score(y_test, final_preds)
print("Final test score:", final_score)

Final test score: 477.5415315131301
